In [0]:
%run ./003_common_utils_keycloak

In [0]:
# Definir esquema esperado
schema_dn_user = StructType([
    StructField("parent_group", StringType(), True),
    StructField("grupo", StringType(), True),
    StructField("nombre", StringType(), True),
    StructField("apellido", StringType(), True),
    StructField("email", StringType(), True),
    StructField("enabled", BooleanType(), True),
    StructField("verified", BooleanType(), True)
])

In [0]:
# Crear lector
reader = PostgresReader(
    scope="secret-storeview",
    username_key="username-keycloak-db",
    password_key="password-keycloak-db",
    hostname="psql-dn-keycloak-restore.postgres.database.azure.com",
    port=5432,
    database="keycloak"
)

In [0]:
# Leer tabla
df = reader.read_table("dn_user", schema=schema_dn_user)

In [0]:
# Validar esquema
validator = SchemaValidator(spark)
validator.validate(reader.jdbc_url, "dn_user", reader.connection_properties, schema_dn_user)

In [0]:
# Transformar
transformer = UserTransformer(df)
df_transformed = (
    transformer
    .add_solution_column()
    .add_user_type_column()
    .clean_column_names()
    .get_df()
)

In [0]:
# Escribir en bronze
writer = DeltaWriter(base_path="/mnt/bronze/keycloak")
writer.write(
    df=df_transformed,
    table_name="dn_user",
    database="bronze",
    mode="overwrite",
)

In [0]:
# Verificar
df_val = spark.table("bronze.dn_user")
display(df_val.limit(10))

In [0]:
# Liberar memoria
df = df_transformed = transformer = reader = writer = None
del df, df_transformed, transformer, reader, writer